In [ ]:
!pip install pyomo
!apt-get install -y -qq glpk-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.2 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 120828 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [15]:
# Defining concrete model object
model = pyo.ConcreteModel()

# Sets
model.i = pyo.RangeSet(1,3)
model.j = pyo.RangeSet(1,4)

# Parameters defined with sets
model.S = pyo.Param(model.i, initialize = {1:35,2:50,3:40})
S = model.S
model.D = pyo.Param(model.j, initialize = {1:45,2:20,3:30,4:30})
D = model.D
model.P = pyo.Param(model.i,model.j, initialize ={(1,1):8,(1,2):6,(1,3):10,(1,4):9,(2,1):9,(2,2):12,(2,3):13,(2,4):7,(3,1):14,(3,2):9,(3,3):16,(3,4):5})
P = model.P

# Variables
model.x = pyo.Var(model.i,model.j, within = pyo.NonNegativeReals)
x = model.x

# Objective function
def objective_rule(model):
  return sum(sum(P[i,j]*x[i,j] for i in model.i) for j in model.j)
model.objec = pyo.Objective(rule = objective_rule)

# Constraint
# there is better way to formulate i index in function, with for loop, not like this hardcoding...
def constraint1(model,i):
  return sum(x[1,j] for j in model.j) <= S[1]
model.const1 = pyo.Constraint(rule = constraint1)

def constraint2(model,i):
  return sum(x[2,j] for j in model.j) <= S[2]
model.const2 = pyo.Constraint(rule = constraint2)

def constraint3(model,i):
  return sum(x[3,j] for j in model.j) <= S[3]
model.const3 = pyo.Constraint(rule = constraint3)

def constraint4(model,j):
  return sum(x[i,1] for i in model.i) >= D[1]
def constraint5(model,j):
  return sum(x[i,2] for i in model.i) >= D[2]
model.const5 = pyo.Constraint(rule = constraint5)

def constraint6(model,j):
  return sum(x[i,3] for i in model.i) >= D[3]
model.const6 = pyo.Constraint(rule = constraint6)

def constraint7(model,j):
  return sum(x[i,4] for i in model.i) >= D[4]
model.const7 = pyo.Constraint(rule = constraint7)

Solver = SolverFactory('glpk')
results = Solver.solve(model)

# Show results
print(results)
print('Objective function =',model.objec())
for i in model.i:
  for j in model.j:
    print('Electicity sent from plant', i, 'to the city', j,'is', x[i,j]())


Problem: 
- Name: unknown
  Lower bound: 615.0
  Upper bound: 615.0
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 12
  Number of nonzeros: 21
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0036759376525878906
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective function = 615.0
Electicity sent from plant 1 to the city 1 is 0.0
Electicity sent from plant 1 to the city 2 is 20.0
Electicity sent from plant 1 to the city 3 is 15.0
Electicity sent from plant 1 to the city 4 is 0.0
Electicity sent from plant 2 to the city 1 is 0.0
Electicity sent from plant 2 to the city 2 is 0.0
Electicity sent from plant 2 to the city 3 is 15.0
Electicity sent from plant 2 to the city 4 is 0.0
Electicity sent from plant 3 to the city 1 is 0.0
Electicity sent from plant 3 to the city 2